# Build Your First LLM App with LCEL

In this quickstart, we’ll walk through building a simple LLM-powered application using LangChain. The app will take English text and translate it into another language.

This is a beginner-friendly project — it’s just one LLM call with a prompt. But don’t be fooled! Even with something this simple, you’ll get a solid introduction to LangChain. Many powerful applications are built from just prompts and LLM calls.

By the end of this tutorial, you’ll understand how to:

Work with language models

Create and use PromptTemplates and OutputParsers

Combine components with LangChain Expression Language (LCEL)

Debug and trace your app using LangSmith

Deploy your app with LangServe

Direct invoke = talk to model

Parser = clean reply

Chain = connect steps

Prompt messages = see inputs

Final chain = full automated pipeline

# LCEL = LangChain Expression Language

It’s like a mini programming language inside LangChain that makes it easy to connect and compose different components (models, prompts, parsers, retrievers, etc.) into one workflow.

Think of it as “glue code” for LLM apps.

# What LCEL does:

Lets you chain components together (e.g., prompt → model → parser → output).

Makes your pipeline declarative (you describe what should happen, not write lots of boilerplate code).

Adds built-in tracing, streaming, and async support without extra code.

Keeps your app modular so you can swap out models, prompts, or parsers easily.

In [4]:
pip install openai


     ---------------------------------------- 0.0/955.5 kB ? eta -:--:--
     -- ------------------------------------ 71.7/955.5 kB 2.0 MB/s eta 0:00:01
     ------------------ ------------------- 471.0/955.5 kB 5.9 MB/s eta 0:00:01
     -------------------------------------  952.3/955.5 kB 8.6 MB/s eta 0:00:01
     -------------------------------------- 955.5/955.5 kB 6.8 MB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached jiter-0.11.0-cp310-cp310-win_amd64.whl (204 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
#groq_api_key

In [7]:
pip install langchain_openai

  Using cached langchain_openai-0.3.33-py3-none-any.whl (74 kB)
     ---------------------------------------- 0.0/948.6 kB ? eta -:--:--
     -- ------------------------------------ 71.7/948.6 kB 2.0 MB/s eta 0:00:01
     ---------------------- --------------- 553.0/948.6 kB 7.0 MB/s eta 0:00:01
     -------------------------------------  942.1/948.6 kB 8.5 MB/s eta 0:00:01
     -------------------------------------- 948.6/948.6 kB 6.7 MB/s eta 0:00:00
  Using cached tiktoken-0.11.0-cp310-cp310-win_amd64.whl (884 kB)
  Using cached regex-2025.9.18-cp310-cp310-win_amd64.whl (276 kB)
  Attempting uninstall: openai
    Found existing installation: openai 2.0.0
    Uninstalling openai-2.0.0:
      Successfully uninstalled openai-2.0.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [9]:
# 🔑 API Keys
openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [10]:

# ✅ Initialize Models
# OpenAI GPT
openai_model = ChatOpenAI(model = "gpt-4o-min", api_key =openai_api_key )


In [11]:
# Groq with Llama3 (you can also try gemma2-9b-it or mistral-7b)
#groq_model = ChatGroq(model="llama3-8b-8192", api_key=groq_api_key)
groq_model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)


In [12]:
# -------------------------
# 🟢 Example 1: Direct messages
messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result = groq_model.invoke(messages)
print("Groq Result:", result.content)

Groq Result: **Hello**  - Bonjour

**How are you?** - Comment allez-vous? 

(You can also use the more informal: **Comment ça va?**) 



Let me know if you have any other phrases you'd like me to translate!



In [13]:
# -------------------------
# 🟢 Example 2: Using Parser
parser= StrOutputParser()
parsed_result = parser.invoke(result)
print("parser:", parsed_result)


parser: **Hello**  - Bonjour

**How are you?** - Comment allez-vous? 

(You can also use the more informal: **Comment ça va?**) 



Let me know if you have any other phrases you'd like me to translate!



In [14]:

# -------------------------
# 🟢 Example 3: Using LCEL chaining
chain = groq_model | parser
print("chained", chain.invoke(messages))

chained Bonjour, comment allez-vous ? 



In [15]:
generic_template = "Translate the following into {language} :"
prompt = ChatPromptTemplate.from_messages([
    ("system", generic_template),
    ("user", "{text}")
])


In [16]:

# -------------------------
# 🟢 Example 4: Prompt Templates + LCEL
generic_template = "Translate the following into {language} :"
prompt = ChatPromptTemplate.from_messages([
    
    ("system",generic_template),
    ("user","{text}")
    
])



In [17]:
result = prompt.invoke({"language" : "French", "text":"Hello"})
print("prompt messages:", result.to_messages())
print("____________________________________________")

# Full chain with prompt + model + parser


prompt messages: [SystemMessage(content='Translate the following into French :', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]
____________________________________________


In [18]:
chain = prompt | groq_model | parser
print("final chain:", chain.invoke({"language": "japanese" , "text": "Hello"}))

final chain: こんにちは (Konnichiwa) 

